# Covarience and Contravarience

In [4]:
showInt :: Int -> String
showInt = show

floorInt :: Double -> Int
floorInt = floor

maybeInt :: Maybe Int
maybeInt = Just 5

fmapMaybe :: (a -> b) -> Maybe a -> Maybe b
fmapMaybe = fmap

fmapMaybe showInt maybeInt

fmapMaybe floorInt maybeInt

Just "5"

<interactive>:1:20: error:
    • Couldn't match type ‘Int’ with ‘Double’
      Expected type: Maybe Double
        Actual type: Maybe Int
    • In the second argument of ‘fmapMaybe’, namely ‘maybeInt’
      In the expression: fmapMaybe floorInt maybeInt
      In an equation for ‘it’: it = fmapMaybe floorInt maybeInt

We are not able to do `fmapMaybe floorInt maybeInt` because we need a function that take an `Int` as input but `floorInt` returns an `Int` as output.
`Maybe` is covariant on it's argument type; the `Functor` is a covariant function.

## A non-covariant data type

In [8]:
newtype MakeStr a = MakeStr { makeStr :: a -> String }

showInt :: MakeStr Int
showInt = MakeStr show

makeStr showInt 5

"5"

What if we want to add 3 to the number before making the string?

In [9]:
plus3ShowInt :: MakeStr Int
plus3ShowInt = MakeStr (show . (+3))

makeStr plus3ShowInt 5

"8"

This approach is 'non-compositional' because we didn't reuse our `showInt` function. We'd ideally like to just apply more functions to this data structure. Let's first write that up without any typeclasses.

In [13]:
mapMakeStr :: (b -> a) -> MakeStr a -> MakeStr b
mapMakeStr f (MakeStr g) = MakeStr (g . f)

plus3ShowInt :: MakeStr Int
plus3ShowInt = mapMakeStr (+3) showInt

makeStr plus3ShowInt 5

"8"

This type of mapping is exactly what we use `Functor` for, so let's write an instance:

In [12]:
instance Functor MakeStr where
  fmap f (MakeStr g) = MakeStr (g . f)

<interactive>:2:37: error:
    • Couldn't match type ‘b’ with ‘a’
      ‘b’ is a rigid type variable bound by
        the type signature for:
          fmap :: forall a b. (a -> b) -> MakeStr a -> MakeStr b
        at <interactive>:2:3-6
      ‘a’ is a rigid type variable bound by
        the type signature for:
          fmap :: forall a b. (a -> b) -> MakeStr a -> MakeStr b
        at <interactive>:2:3-6
      Expected type: a -> a
        Actual type: a -> b
    • In the second argument of ‘(.)’, namely ‘f’
      In the first argument of ‘MakeStr’, namely ‘(g . f)’
      In the expression: MakeStr (g . f)
    • Relevant bindings include
        g :: a -> String (bound at <interactive>:2:19)
        f :: a -> b (bound at <interactive>:2:8)
        fmap :: (a -> b) -> MakeStr a -> MakeStr b (bound at <interactive>:2:3)

To understand why this failed, lets examine the type for `fmap` against our `mapMakeStr` function:

`fmap :: (a -> b) -> MakeStr a -> MakeStr b`
`mapMakeStr :: (b -> a) -> MakeStr a -> MakeStr b`

In `MakeStr a`, `a` is the type of the input of the function (`String` is the return type) and so `map` in this context means that we are transforming that input before it reaches the next operation (which takes an `a`). So we want the input to be a `b` but in order to compose it with `MakeStr a`, we need a function that is `b -> a`.

Another way to look at it is that we are wrapping additional functions around the data structure, and not chaining the output of an initial function into a new function which is what `fmap` expects.

- Covariance means that both the original and the lifted functions point in the same direction.
- Contravarience means that the original and the lifted functions point in opposite directions.

This is what it means when we refer to the `Functor` typeclass as a covariant functor.

In [17]:
import Data.Functor.Contravariant

instance Contravariant MakeStr where
  contramap f (MakeStr g) = MakeStr (g . f)
  
plus3ShowInt = contramap (+3) showInt

makeStr plus3ShowInt 5

"8"

## Filtering with `Predicate`

In [22]:
greaterThanThree :: Int -> Bool
greaterThanThree = (> 3)

lengthGTThree :: [a] -> Bool
lengthGTThree = greaterThanThree . length

englishGTThree :: Int -> Bool
englishGTThree = lengthGTThree . english

english :: Int -> String
english 1 = "one"
english 2 = "two"
english 3 = "three"
english 4 = "four"
english 5 = "five"
english 6 = "six"
english 7 = "seven"
english 8 = "eight"
english 9 = "nine"
english 10 = "ten"

filter englishGTThree [1..10]

[3,4,5,7,8,9]

In [23]:
import Data.Functor.Contravariant

greaterThanThree :: Predicate Int
greaterThanThree = Predicate (> 3)

lengthGTThree :: Predicate [a]
lengthGTThree = contramap length greaterThanThree

englishGTThree :: Predicate Int
englishGTThree = contramap english lengthGTThree

english :: Int -> String
english 1 = "one"
english 2 = "two"
english 3 = "three"
english 4 = "four"
english 5 = "five"
english 6 = "six"
english 7 = "seven"
english 8 = "eight"
english 9 = "nine"
english 10 = "ten"

filter (getPredicate englishGTThree) [1..10]

[3,4,5,7,8,9]

## Bifunctor and Profunctor

In [26]:
class Bifunctor p where
  bimap :: (a -> b) -> (c -> d) -> p a c -> p b d
  
class Profunctor p where
  dimap :: (b -> a) -> (c -> d) -> p a c -> p b d

We can use what we now know to understand the formal definition of these

    Bifunctor: Intuitively it is a bifunctor where both the first and second arguments are covariant.

    Profunctor: Intuitively it is a bifunctor where the first argument is contravariant and the second argument is covariant.

These are both bifunctors since they take two type parameters. They both treat their second parameter in the same way: covariantly. However, the first parameter is treated differently by the two: Bifunctor is covariant, and Profunctor is contravariant.

In [33]:
instance Bifunctor Either where
  bimap f _ (Left a) = Left $ f a
  bimap _ g (Right b) = Right $ g b
  
instance Bifunctor (,) where
  bimap f g (a, b) = (f a, g b)
  
instance Profunctor (->) where
  dimap f g x = g . x . f

## Bivariant and Invariant

Bivariant refers to types that are both Covariant and Contravariant. Invariant refers to types that are neither. Only phantoms (where the type doesn't actually exist) can be Bivariant. example:

In [35]:
data Phantom a = Phantom

instance Functor Phantom where
  fmap f Phantom = Phantom
instance Contravariant Phantom where
  contramap f Phantom = Phantom

Invariance occurs when a type parameter is used in two places and in different positions:

In [36]:
data ToFrom a = ToFrom (Int -> a) (a -> Int)

or if a type parameter is used in a type that is itself invariant:

In [37]:
newtype ToFromWrapper a = ToFromWrapper (ToFrom a)

or in special types like references such as `IORef a`

## Positive and Negative position

In [1]:
data WithInt a = WithInt (a -> Int)
data MakeInt a = MakeInt (Int -> a)

We know that `WithInt` is Contravariant and MakeInt is Covariant because `a` is being used as an input or output.

The compiler can tell us this by trying to derive `Functor`.

In [4]:
:ext DeriveFunctor

newtype WithInt a = WithInt (a -> Int) deriving Functor

<interactive>:1:49: error:
    • Can't make a derived instance of ‘Functor WithInt’: Constructor ‘WithInt’ must not use the type variable in a function argument
    • In the newtype declaration for ‘WithInt’

- Positive position is the output / result / range / codomain
- Negative position is the input / argument / domain

If a type variable is in the positive position, the data type is covariant with that variable. Vice versa for negative / contravariant.

But why use the terms positive and negative?

In [23]:
type Callback a = a -> IO ()
-- newtype CallbackRunner a = CallbackRunner (Callback a -> IO ())
newtype CallbackRunner a = CallbackRunner { runCallback :: (a -> IO ()) -> IO () }

Is `a` covariant or contravariant? Your instinct may be that `a` is a function parameter therefore it's contravariant.

If we're just dealing with `a -> IO ()` then we know that a is contravarient (negative). But what about when we wrap this as the input to another function?

In [20]:
:m System.Random

supplyRandom :: CallbackRunner Int
supplyRandom = CallbackRunner $ \callback -> do
  int <- randomRIO (0, 10)
  callback int
  
runCallback supplyRandom print

10

We can see that `supplyRandom` is actually producing an `Int`, so it's covariant. Let's use the notion of positive and negative positions to better explain whats happening.

In `a -> IO ()`, `a` is negative and in `(a -> IO ()) -> IO ()` the `Callback a` is negative, so we follow multiplication rules and say the a negative times a negative is positive. This explains why `a` is covariant and we can derive a `Functor` instance.

In [22]:
newtype CallbackRunner a = CallbackRunner { runCallback :: (a -> IO ()) -> IO () } deriving Functor

Let's try deriving the `Functor` instance manually.

In [29]:
instance Functor CallbackRunner where
  fmap f (CallbackRunner runner) = 
    CallbackRunner $ \callback -> 
      runner $ callback . f

In [44]:
newtype E1 a = E1 (a -> ()) -- negative
newtype E2 a = E2 (a -> () -> ()) -- negative
newtype E3 a = E3 ((a -> ()) -> ()) -- positive
newtype E4 a = E4 ((a -> () -> ()) -> ()) -- positive
newtype E5 a = E5 ((() -> () -> a) -> ()) -- negative

-- trickier:
newtype E6 a = E6 ((() -> a -> a) -> ()) -- negative
newtype E7 a = E7 ((() -> () -> a) -> a) -- negative
newtype E8 a = E8 ((() -> a -> ()) -> a) -- positive
newtype E9 a = E9 ((() -> () -> ()) -> ()) -- bivariant

## Lifting `IO` to `MonadIO`
Let's look at something seemingly unrelated.

In [47]:
import System.IO

:t openFile

openFile :: FilePath -> IOMode -> IO Handle

To use this from a monad transformer, we use the `MonadIO` typeclass constraint and it's `liftIO` function

In [56]:
import Control.Monad.IO.Class

liftedOpenFile :: MonadIO m => FilePath -> IOMode -> m Handle
liftedOpenFile = (liftIO .) . openFile

It's better of course to use `withFile` so we don't have to manually close the file

In [57]:
:t withFile

withFile :: forall r. FilePath -> IOMode -> (Handle -> IO r) -> IO r

In [64]:
liftedWithFile :: MonadIO m => FilePath -> IOMode -> (Handle -> m r) -> m r
liftedWithFile fp mode cb = liftIO (withFile fp mode (liftIO . cb)) -- can't do it!

<interactive>:2:64: error:
    • Couldn't match type ‘m’ with ‘IO’
      ‘m’ is a rigid type variable bound by
        the type signature for:
          liftedWithFile :: forall (m :: * -> *) r. MonadIO m => FilePath -> IOMode -> (Handle -> m r) -> m r
        at <interactive>:1:1-75
      Expected type: Handle -> IO r
        Actual type: Handle -> m r
    • In the second argument of ‘(.)’, namely ‘cb’
      In the third argument of ‘withFile’, namely ‘(liftIO . cb)’
      In the first argument of ‘liftIO’, namely ‘(withFile fp mode (liftIO . cb))’
    • Relevant bindings include
        cb :: Handle -> m r (bound at <interactive>:2:24)
        liftedWithFile :: FilePath -> IOMode -> (Handle -> m r) -> m r (bound at <interactive>:2:1)

The reason it can't be done, at least not directly, is that `IO` appears in both negative and positive positions whereas with `openFile` it is only in the positive position.